In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np
import re
from utils import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [2]:
files_dir = "../data/impresso-data"
archives = os.listdir(files_dir)
date = "1968"

* Extract interesting archives

In [3]:

gdl_1968 = extract_particular_files(archives, '(.*197[0-2].*)|(.*196[5-9].*)', 'GDL')      
jdg_1968 = extract_particular_files(archives, '(.*197[0-2].*)|(.*196[5-9].*)', 'JDG') 

* Extract content  and preprocessing

In [4]:
df_all, df_gdl, df_jdg = run(gdl_1968, jdg_1968, date, files_dir)

GDL-1965.jsonl.bz2 : 51962 articles à extraire
GDL-1966.jsonl.bz2 : 55456 articles à extraire
GDL-1967.jsonl.bz2 : 55126 articles à extraire
GDL-1968.jsonl.bz2 : 51434 articles à extraire
GDL-1969.jsonl.bz2 : 50467 articles à extraire
GDL-1970.jsonl.bz2 : 58656 articles à extraire
GDL-1971.jsonl.bz2 : 54695 articles à extraire
GDL-1972.jsonl.bz2 : 47024 articles à extraire
JDG-1965.jsonl.bz2 : 44262 articles à extraire
JDG-1966.jsonl.bz2 : 46881 articles à extraire
JDG-1967.jsonl.bz2 : 44754 articles à extraire
JDG-1968.jsonl.bz2 : 45200 articles à extraire
JDG-1969.jsonl.bz2 : 47098 articles à extraire
JDG-1970.jsonl.bz2 : 45544 articles à extraire
JDG-1971.jsonl.bz2 : 44609 articles à extraire
JDG-1972.jsonl.bz2 : 45846 articles à extraire


,index,mag,date,page,text
668296,267773,JDG,1971-07-12,10,"Les 36 processions de l'Ulster •• Dimanche, tout s'est passé dans l'ordre, mais ce n'est qu'appa..."


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,GDL,1965-07-07,1,NaN,Paris met en veilleuse le Marché commun et la ignorera dorénavant ommission Hallstein La France ...,7217
1,GDL,1965-07-07,2,NaN,"Raymond Poulidor talonne F. Gimondi et met k. o. un autre favori, Gianni Motta Eblouissant vainq...",6467
2,GDL,1965-07-07,3,NaN,La prof an du manquee de Necker Vevey : nouvel épisode du Banquier sans visage » Le Tribunal cri...,7427
3,GDL,1965-07-07,4,NaN,"UN ROMAN COMPLET Et puis soudain, elle vit quelqu'un, Fernand, son père. Les yeux déjà à demi fe...",9367
4,GDL,1965-07-07,5,NaN,"Un troisième tunnel du Gothard ? Non, niais une ligne Todi-Greina L'ÉQUIPEMENT FERROVIAIRE DE LA...",6830


Length df: 224      820
71       804
131      740
200      728
201      723
        ... 
14054      1
10643      1
12396      1
14445      1
16035      1
Name: length, Length: 15421, dtype: int64


...

,mag,date,page,ppage,text,length
0,GDL,1965-07-07,1,NaN,Paris met en veilleuse le Marché commun et la ignorera dorénavant ommission Hallstein La France ...,7217
1,GDL,1965-07-07,2,NaN,"Raymond Poulidor talonne F. Gimondi et met k. o. un autre favori, Gianni Motta Eblouissant vainq...",6467
2,GDL,1965-07-07,3,NaN,La prof an du manquee de Necker Vevey : nouvel épisode du Banquier sans visage » Le Tribunal cri...,7427
3,GDL,1965-07-07,4,NaN,"UN ROMAN COMPLET Et puis soudain, elle vit quelqu'un, Fernand, son père. Les yeux déjà à demi fe...",9367
4,GDL,1965-07-07,5,NaN,"Un troisième tunnel du Gothard ? Non, niais une ligne Todi-Greina L'ÉQUIPEMENT FERROVIAIRE DE LA...",6830


Length df: 71       658
68       581
224      525
131      516
61       485
        ... 
21589      1
9051       1
10842      1
7407       1
12009      1
Name: length, Length: 11718, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,JDG,1965-07-07,1,NaN,"Algérie, an III L'Alcéric a fêté le troisième anniversaire de son indépcndance avec la simplicit...",3680
1,JDG,1965-07-07,2,NaN,le Conseil fédéral rend compte de l'application et des effets des mesures prises pour freiner le...,7857
2,JDG,1965-07-07,3,NaN,"« Les Elals-Unis leront ce Qu'illaudra Honolulu, 6.-(AFP) De retour d'une tournée d'inspection a...",3179
3,JDG,1965-07-07,4,NaN,"Financer les idées nouvelles En décembre 1963 a été créée à Luxembourg, avec un capital de 125 m...",6774
4,JDG,1965-07-07,5,NaN,Chronlaue des Bourse. Bourses européennes Devant l'absence d'indications en provenance do New-Yo...,5346


Length df: 201      326
144      325
214      317
74       316
206      316
        ... 
16712      1
22169      1
17819      1
19866      1
12665      1
Name: length, Length: 13583, dtype: int64


* Keywords filtering

In [5]:
keywords_1968 = ['nucléaire', 'nucleaire', "prolifération", 'proliferation', "arme", 'armes', "bombe", 'bombes', "technologie", 'technologies', "traité", 'traite', "international", 
            "guerre", 'signer', 'signe', 'Londres', 'Moscou', 'Washington', 'ratification', '1976', '1977']
keywords_1962 =['nucléaire', 'Cuba', 'guerre', 'tension','risque', 'bombe', 'guerre nucléaire']
unwanted = ['tableau hebdomadaire']#, '(\/(\s\d*)*\s)*']

cleaned_files_dir = "../data/filtered/"
archives = os.listdir(files_dir)

In [6]:
#df_all = load_corpus(cleaned_files_dir + "cleaned_all" + date +".json.bz2")
#df_gdl = load_corpus(cleaned_files_dir + "cleaned_gdl" + date +".json.bz2")
#df_jdg = load_corpus(cleaned_files_dir + "cleaned_jdg" + date +".json.bz2")



In [7]:
df_all_ = keywords_filtering(df_all, keywords_1968)
df_all_ = unwanted_filtering(df_all_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(94421, 29)

=====UNWANTED====


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_nucleaire,keyword_prolifération,keyword_proliferation,keyword_arme,keyword_armes,keyword_bombe,keyword_bombes,keyword_technologie,keyword_technologies,keyword_traité,keyword_traite,keyword_international,keyword_guerre,keyword_signer,keyword_signe,keyword_Londres,keyword_Moscou,keyword_Washington,keyword_ratification,keyword_1976,keyword_1977,garbage
0,GDL,1965-07-07,1,NaN,Paris met en veilleuse le Marché commun et la ignorera dorénavant ommission Hallstein La France ...,7217,2,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,GDL,1965-07-07,3,NaN,La prof an du manquee de Necker Vevey : nouvel épisode du Banquier sans visage » Le Tribunal cri...,7427,3,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,GDL,1965-07-07,4,NaN,"UN ROMAN COMPLET Et puis soudain, elle vit quelqu'un, Fernand, son père. Les yeux déjà à demi fe...",9367,12,0,0,0,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,GDL,1965-07-07,5,NaN,"Un troisième tunnel du Gothard ? Non, niais une ligne Todi-Greina L'ÉQUIPEMENT FERROVIAIRE DE LA...",6830,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
7,GDL,1965-07-07,8,NaN,"Etats-Uni s reconnaissent l e nouvea u régim e algérie n -WASHINGTON, — « Les Etats-Unis reconna...",1782,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


'After keyword filtered: '

(94389, 29)

In [8]:
df_gdl_ = keywords_filtering(df_gdl, keywords_1968)
df_gdl_ = unwanted_filtering(df_gdl_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(38954, 29)

=====UNWANTED====


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_nucleaire,keyword_prolifération,keyword_proliferation,keyword_arme,keyword_armes,keyword_bombe,keyword_bombes,keyword_technologie,keyword_technologies,keyword_traité,keyword_traite,keyword_international,keyword_guerre,keyword_signer,keyword_signe,keyword_Londres,keyword_Moscou,keyword_Washington,keyword_ratification,keyword_1976,keyword_1977,garbage
0,GDL,1965-07-07,1,NaN,Paris met en veilleuse le Marché commun et la ignorera dorénavant ommission Hallstein La France ...,7217,2,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,GDL,1965-07-07,3,NaN,La prof an du manquee de Necker Vevey : nouvel épisode du Banquier sans visage » Le Tribunal cri...,7427,3,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,GDL,1965-07-07,4,NaN,"UN ROMAN COMPLET Et puis soudain, elle vit quelqu'un, Fernand, son père. Les yeux déjà à demi fe...",9367,12,0,0,0,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,GDL,1965-07-07,5,NaN,"Un troisième tunnel du Gothard ? Non, niais une ligne Todi-Greina L'ÉQUIPEMENT FERROVIAIRE DE LA...",6830,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
7,GDL,1965-07-07,8,NaN,"Etats-Uni s reconnaissent l e nouvea u régim e algérie n -WASHINGTON, — « Les Etats-Unis reconna...",1782,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


'After keyword filtered: '

(38949, 29)

In [9]:
df_jdg_ = keywords_filtering(df_jdg, keywords_1968)
df_jdg_ = unwanted_filtering(df_jdg_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(55467, 29)

=====UNWANTED====


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_nucleaire,keyword_prolifération,keyword_proliferation,keyword_arme,keyword_armes,keyword_bombe,keyword_bombes,keyword_technologie,keyword_technologies,keyword_traité,keyword_traite,keyword_international,keyword_guerre,keyword_signer,keyword_signe,keyword_Londres,keyword_Moscou,keyword_Washington,keyword_ratification,keyword_1976,keyword_1977,garbage
0,JDG,1965-07-07,1,NaN,"Algérie, an III L'Alcéric a fêté le troisième anniversaire de son indépcndance avec la simplicit...",3680,5,0,0,0,0,1,1,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0
1,JDG,1965-07-07,2,NaN,le Conseil fédéral rend compte de l'application et des effets des mesures prises pour freiner le...,7857,5,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0
3,JDG,1965-07-07,4,NaN,"Financer les idées nouvelles En décembre 1963 a été créée à Luxembourg, avec un capital de 125 m...",6774,4,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,0
4,JDG,1965-07-07,5,NaN,Chronlaue des Bourse. Bourses européennes Devant l'absence d'indications en provenance do New-Yo...,5346,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0
11,JDG,1965-07-07,12,NaN,De Gaulle face au Marché commun la France applique a ses partenaires la tactique de l'escalade P...,9416,2,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


'After keyword filtered: '

(55440, 29)

* Export iramutek

In [10]:
display(df_all.head())
display(df_gdl.head())
display(df_jdg.head())

,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_nucleaire,keyword_prolifération,keyword_proliferation,keyword_arme,keyword_armes,keyword_bombe,keyword_bombes,keyword_technologie,keyword_technologies,keyword_traité,keyword_traite,keyword_international,keyword_guerre,keyword_signer,keyword_signe,keyword_Londres,keyword_Moscou,keyword_Washington,keyword_ratification,keyword_1976,keyword_1977
0,GDL,1965-07-07,1,NaN,Paris met en veilleuse le Marché commun et la ignorera dorénavant ommission Hallstein La France ...,7217,2,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,GDL,1965-07-07,2,NaN,"Raymond Poulidor talonne F. Gimondi et met k. o. un autre favori, Gianni Motta Eblouissant vainq...",6467,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GDL,1965-07-07,3,NaN,La prof an du manquee de Necker Vevey : nouvel épisode du Banquier sans visage » Le Tribunal cri...,7427,3,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,GDL,1965-07-07,4,NaN,"UN ROMAN COMPLET Et puis soudain, elle vit quelqu'un, Fernand, son père. Les yeux déjà à demi fe...",9367,12,0,0,0,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,GDL,1965-07-07,5,NaN,"Un troisième tunnel du Gothard ? Non, niais une ligne Todi-Greina L'ÉQUIPEMENT FERROVIAIRE DE LA...",6830,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_nucleaire,keyword_prolifération,keyword_proliferation,keyword_arme,keyword_armes,keyword_bombe,keyword_bombes,keyword_technologie,keyword_technologies,keyword_traité,keyword_traite,keyword_international,keyword_guerre,keyword_signer,keyword_signe,keyword_Londres,keyword_Moscou,keyword_Washington,keyword_ratification,keyword_1976,keyword_1977
0,GDL,1965-07-07,1,NaN,Paris met en veilleuse le Marché commun et la ignorera dorénavant ommission Hallstein La France ...,7217,2,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,GDL,1965-07-07,2,NaN,"Raymond Poulidor talonne F. Gimondi et met k. o. un autre favori, Gianni Motta Eblouissant vainq...",6467,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GDL,1965-07-07,3,NaN,La prof an du manquee de Necker Vevey : nouvel épisode du Banquier sans visage » Le Tribunal cri...,7427,3,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,GDL,1965-07-07,4,NaN,"UN ROMAN COMPLET Et puis soudain, elle vit quelqu'un, Fernand, son père. Les yeux déjà à demi fe...",9367,12,0,0,0,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,GDL,1965-07-07,5,NaN,"Un troisième tunnel du Gothard ? Non, niais une ligne Todi-Greina L'ÉQUIPEMENT FERROVIAIRE DE LA...",6830,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_nucleaire,keyword_prolifération,keyword_proliferation,keyword_arme,keyword_armes,keyword_bombe,keyword_bombes,keyword_technologie,keyword_technologies,keyword_traité,keyword_traite,keyword_international,keyword_guerre,keyword_signer,keyword_signe,keyword_Londres,keyword_Moscou,keyword_Washington,keyword_ratification,keyword_1976,keyword_1977
0,JDG,1965-07-07,1,NaN,"Algérie, an III L'Alcéric a fêté le troisième anniversaire de son indépcndance avec la simplicit...",3680,5,0,0,0,0,1,1,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0
1,JDG,1965-07-07,2,NaN,le Conseil fédéral rend compte de l'application et des effets des mesures prises pour freiner le...,7857,5,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0
2,JDG,1965-07-07,3,NaN,"« Les Elals-Unis leront ce Qu'illaudra Honolulu, 6.-(AFP) De retour d'une tournée d'inspection a...",3179,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,JDG,1965-07-07,4,NaN,"Financer les idées nouvelles En décembre 1963 a été créée à Luxembourg, avec un capital de 125 m...",6774,4,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0
4,JDG,1965-07-07,5,NaN,Chronlaue des Bourse. Bourses européennes Devant l'absence d'indications en provenance do New-Yo...,5346,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0


In [12]:
creating_iramutek_file(df_all_, "../data/iramutek/corpus_all" + date +".txt")
creating_iramutek_file(df_jdg_, "../data/iramutek/corpus_jdg" + date +".txt")
creating_iramutek_file(df_gdl_, "../data/iramutek/corpus_gdl" + date +".txt")